In [11]:
# -*- coding: utf-8 -*-
import numpy as np
import copy
import time
import sys
import os
import json
from operator import itemgetter

openlist = []    # 存放待扩展的节点
closelist = []   # 存放已扩展的节点
goal = {}        # 目标节点

def get_location(mat, element):
# 获取element在mat中的位置,返回坐标i, j
    row_num = mat.shape[0]
    col_num = mat.shape[1]
    
    for i in range(row_num):
        for j in range(col_num):
            if element == mat[i][j]:
                return i, j

def get_actions(mat):
# 获取mat中的0元素可以移动的下一个位置，返回可移动动作的列表
    row_num = mat.shape[0]
    col_num = mat.shape[1]
    
    (x, y) = get_location(mat, 0)
    action = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # action：朝4个方向分别移动1格
    
    # 0在边缘:可以减少可执行的action
    if x == 0:
        action.remove((-1, 0))
    if y == 0:
        action.remove((0, -1))
    if x == row_num - 1:
        action.remove((1, 0))
    if y == col_num - 1:
        action.remove((0, 1))
        
    return list(action)

def result(mat, action):
# 0元素执行action，返回结果
    (x, y) = get_location(mat, 0)
    (a, b) = action
    
    # 0元素执行action，两个元素交换位置
    n = mat[x+a][y+b]
    s = copy.deepcopy(mat)
    s[x+a][y+b] = 0
    s[x][y] = n
     
    return s
    
def get_MDis(mat1, mat2):
# 计算两个矩阵的曼哈顿距离,mat1为目标矩阵,mat2为当前矩阵
    row_num = mat1.shape[0]
    col_num = mat1.shape[1]
    dis  = 0
    
    for i in range(row_num):
        for j in range(col_num):
            if mat1[i][j] != mat2[i][j] and mat2[i][j] != 0:
                k, m = get_location(mat1, mat2[i][j])
                d = abs(i - k) + abs(j - m)            # 计算曼哈顿距离
                dis += d
                
    return dis

def update(p, actions, step):
# actions为当前节点的可执行动作列表,p为当前矩阵（即父节点）,step为已走的步数
    children = []                                      # children用来保存父节点的子节点
    for action in actions:
        child = {}
        child['parent'] = p
        child['mat'] = (result(p['mat'], action))
        child['dis'] = get_MDis(goal['mat'], child['mat'])
        child['step'] = step + 1                       # 每扩展一次当前已走距离加1
        child['dis'] = child['dis'] + child['step']    # 更新该节点的f值  f = g+h = step+child[‘dis']                     
        child['action'] = get_actions(child['mat'])
        children.append(child)
    
    return children

def node_sort(nodelist):
# 按当前节点对于目标节点的曼哈顿距离对列表从大到小排序,f值最小的节点在末尾
    return sorted(nodelist, key = itemgetter('dis'), reverse=True)


def get_parent(node):
# 获取父节点
    p = node['parent']   
    return p

def read_data(dir_str):
# 从文件中读取矩阵
    A = []
    temp=[]
    with open(dir_str) as fdata:
        while True:
            line=fdata.readline()
            if not line:
                break
            p = []
            temp = line.split(' ')
            for t in temp:
                if t>='0' and t<='9':
                    t = int(t)
                    p.append(t)
            A.append(p)
    return A

In [ ]:
dir = "config.json"
while not os.path.exists(dir):
# 判断文件是否存在
    print("file doesn't exists!")
    sys.exit()
f = open(dir, encoding='utf-8')
setting = json.load(f)
f.close()

print('读取矩阵的行数')
num = int(setting['num'])
print("求解%d-数码问题"%(num ** 2 - 1))
    
print("读取初始矩阵A")
A = read_data(setting['arrayA'])
print(np.array(A))
 
print("读取目标矩阵B")
B = read_data(setting['arrayB'])
print(np.array(B))
    
resultfile = "result_"
resultfile += str(num ** 2 - 1)
resultfile += "digits.txt"
print("保存结果到文件：", resultfile)
    
# 建立目标节点
goal['mat'] = np.array(B)
# 建立初始的父节点
p = {}
p['mat'] = np.array(A)
p['dis'] = get_MDis(goal['mat'], p['mat'])
p['step'] = 0
p['action'] = get_actions(p['mat'])
p['parent'] = {}

if (p['mat'] == goal['mat']).all():
    print("初始矩阵与目标矩阵相同！")
    sys.exit(0)
    
openlist.append(p)           # 将父节点添加到openlist中
    
search_start = time.time()   # 开始计时
while openlist:
    node = openlist.pop()
    closelist.append(node)
    
    # 对当前节点进行update，取当前节点的所有子节点
    children = []
    children = update(node, node['action'], node['step'])
    for child in children:
    # 如果update之后的节点既不在openlist也不在closelist，则将其插入openlist
    # 如果在closelist中说明已被搜索过，舍弃；如果在openlist,比较这两个矩阵的f值，取f小的节点插入openlist
        open_flag = False
        close_flag = False
        j = 0
        for i in range(len(openlist)):
            if (child['mat'] == openlist[i]['mat']).all():
                j = i
                open_flag = True
                break
        for i in range(len(closelist)):
            if(child['mat'] == closelist[i]).all():
                close_flag = True
                break
        if  open_flag == False and close_flag == False :
            openlist.append(child)  
        elif open_flag == True:
            if child['dis'] < openlist[j]['dis']:
                del openlist[j]
                openlist.append(child)
                    
    openlist = node_sort(openlist)
    
    if (node['mat'] == goal['mat']).all():       # 比较当前矩阵和目标矩阵是否相同
        search_end = time.time()                 # 搜索完成，结束计时
         
        # 将结果写入文件并输出
        h = open(resultfile, 'w', encoding='utf-8')
        h.write('搜索树大小：' + str(len(openlist) + len(closelist)) + '\n')
        h.write('closelist：' + str(len(closelist)) + '\n')
        h.write('openlist：' + str(len(openlist)) + '\n')
        h.write('搜索时间：' + str(search_end - search_start) + '\n')
        h.write('搜索步数：' + str(node['step']) + '\n')
        h.write('解的路径：' + '\n')
        # 从目标节点开始回溯，将其父节点存入way中,再依次pop将解途径的节点依次写入文件
        depth = 0
        way = []
        while closelist:
            way.append(node['mat'])
            node = get_parent(node)
            if(node['mat'] == p['mat']).all():
                way.append(node['mat'])
                break
        while way:
            depth += 1
            h.write(str(depth) + '\n')
            h.write(str(way.pop()) + '\n')
        h.close()
        f = open(resultfile,'r',encoding='utf-8')
        print(f.read())
            
        sys.exit(0)

读取矩阵的行数
求解24-数码问题
读取初始矩阵A
[[ 6  1  4 15  5]
 [ 2 17  3  9  0]
 [11 23  8  7 13]
 [16 18 12 19 10]
 [21 22 24 14 20]]
读取目标矩阵B
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24  0]]
保存结果到文件： result_24digits.txt
